In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os


In [5]:
try:
    carpeta_actual = os.path.dirname(os.path.abspath(__file__))
except NameError:
    carpeta_actual = os.getcwd()  # Alternativa para entornos interactivos como Jupyter

carpeta_actual=carpeta_actual.replace('creacion_red_neuronal_y_entrenamiento', 'datos_iniciales_del_sistema')
print(f"Carpeta actual: {carpeta_actual}")



Carpeta actual: c:\Users\dgome\OneDrive\SIN RECUPERAR\Sql\Documentos\SistemaIot\Periferico\modelos_entrenamiento\vehiculo_autonomo\test


In [3]:

ruta_csv = os.path.join(carpeta_actual,'tinyml_training_data_clean.csv')
# 3. Cargar el archivo CSV
if os.path.exists(ruta_csv):
    try:
        data = pd.read_csv(ruta_csv)
        print(f"Archivo CSV cargado correctamente. Filas: {data.shape[0]}, Columnas: {data.shape[1]}")
    except Exception as e:
        print(f"Error al cargar el archivo CSV: {e}")
        exit()
else:
    print(f"El archivo CSV no existe: {ruta_csv}")
    exit()


Archivo CSV cargado correctamente. Filas: 3737, Columnas: 8


In [4]:
data = pd.read_csv(ruta_csv)
# Verificar la existencia de las columnas necesarias
columnas_entrada = ['distancia_izquierda', 'distancia_derecha', 'distancia_izquierda_duration', 'distancia_derecha_duration']
columnas_salida = ['IN1', 'IN2', 'IN3', 'IN4']

for columna in columnas_entrada + columnas_salida:
    if columna not in data.columns:
        print(f"La columna '{columna}' no está presente en el archivo CSV.")
        exit()


In [5]:
# Definir las entradas (X) y las salidas (y)
X = data[columnas_entrada].values
y = data[columnas_salida].values

# Validar si hay datos faltantes
if np.any(np.isnan(X)) or np.any(np.isnan(y)):
    print("Existen valores NaN en los datos de entrada o salida. Por favor, limpie los datos.")
    exit()

# Normalizar las entradas para evitar errores de división por cero
X_max = np.max(X, axis=0)
if np.any(X_max == 0):
    print("Error: Algunos valores máximos de las columnas de entrada son 0, lo que causará división por cero.")
    exit()
X = X / X_max  # Normalizar de 0 a 1

In [6]:
# Dividir los datos en conjuntos de entrenamiento y prueba
try:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Datos divididos en entrenamiento ({X_train.shape[0]}) y prueba ({X_test.shape[0]}).")
except Exception as e:
    print(f"Error al dividir los datos de entrenamiento y prueba: {e}")
    exit()

Datos divididos en entrenamiento (2989) y prueba (748).


In [7]:
# Crear el modelo de la red neuronal
try:
    input_shape = (X_train.shape[1],)  # Determinar automáticamente el número de características de entrada
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=input_shape),        # Entrada con la forma del dataset
        tf.keras.layers.Dense(8, activation='relu'),  # Capa oculta con 8 neuronas
        tf.keras.layers.Dense(4, activation='sigmoid') # Cuatro salidas (para IN1, IN2, IN3, IN4)
    ])
except Exception as e:
    print(f"Error al crear el modelo de la red neuronal: {e}")
    exit()

In [8]:
# Compilar el modelo
try:
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
except Exception as e:
    print(f"Error al compilar el modelo: {e}")
    exit()
# Entrenar el modelo
try:
    history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)
    print("Entrenamiento completado.")
except Exception as e:
    print(f"Error durante el entrenamiento del modelo: {e}")
    exit()

Epoch 1/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1701 - loss: 0.6875 - val_accuracy: 0.1304 - val_loss: 0.6530
Epoch 2/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1461 - loss: 0.6506 - val_accuracy: 0.1221 - val_loss: 0.6415
Epoch 3/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1534 - loss: 0.6444 - val_accuracy: 0.1572 - val_loss: 0.6333
Epoch 4/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1807 - loss: 0.6310 - val_accuracy: 0.1906 - val_loss: 0.6272
Epoch 5/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2357 - loss: 0.6236 - val_accuracy: 0.2274 - val_loss: 0.6215
Epoch 6/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2260 - loss: 0.6191 - val_accuracy: 0.3579 - val_loss: 0.6155
Epoch 7/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3441 - loss: 0.6171 - val_accuracy: 0.3579 - val_loss: 0.6114
Epoch 8/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3574 - loss: 0.6071 - val_accuracy: 0.

In [9]:
# Evaluar el modelo con los datos de prueba
try:
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f'Pérdida de prueba: {test_loss:.4f}, Precisión de prueba: {test_accuracy:.4f}')
except Exception as e:
    print(f"Error al evaluar el modelo: {e}")
    exit()

24/24 - 0s - 2ms/step - accuracy: 0.4853 - loss: 0.5622
Pérdida de prueba: 0.5622, Precisión de prueba: 0.4853


In [10]:
# Generar el archivo de cabecera (.h) para los pesos y sesgos
ruta_h = os.path.join(carpeta_actual, 'pesos_red_neuronal.h')

try:
    with open(ruta_h, 'w') as archivo_h:
        archivo_h.write("#ifndef PESOS_RED_NEURONAL_H\n")
        archivo_h.write("#define PESOS_RED_NEURONAL_H\n\n")
        
        for i, layer in enumerate(model.layers):
            try:
                weights, biases = layer.get_weights()
                
                archivo_h.write(f"// Pesos de la capa {i}\n")
                archivo_h.write(f"static const float PESOS_CAPA_{i}[] = {{\n")
                
                flat_weights = weights.flatten()
                archivo_h.write(", ".join([f"{w:.6f}" for w in flat_weights]))
                archivo_h.write("\n};\n\n")
                
                archivo_h.write(f"// Sesgos de la capa {i}\n")
                archivo_h.write(f"static const float SESGOS_CAPA_{i}[] = {{\n")
                archivo_h.write(", ".join([f"{b:.6f}" for b in biases]))
                archivo_h.write("\n};\n\n")
                
            except Exception as e:
                print(f"Error al obtener los pesos y sesgos de la capa {i}: {e}")
        
        archivo_h.write("#endif // PESOS_RED_NEURONAL_H\n")
        print(f"Archivo de pesos generado exitosamente en: {ruta_h}")
except Exception as e:
    print(f"Error al escribir el archivo de pesos: {e}")
    exit()


Archivo de pesos generado exitosamente en: c:\Users\dgome\OneDrive\SIN RECUPERAR\Sql\Documentos\SistemaIot\Periferico\modelos_entrenamiento\vehiculo_autonomo\pesos_red_neuronal.h
